### Como os indiqué en clase, en esta prueba abierta de temática libre trata de que hagáis un análisis de un dataset que vosotros seleccionéis (del repositorio de statsmodels, del de R o de cualquier otro que encontréis en Internet). Se valorará la presentación de las ideas de análisis, el uso de la herramienta de los Notebooks, gráficos que sean relevantes y de las bibliotecas del stack científico de Python.




### Solo se pide llegar al análisis estadístico básico, no a hacer modelos productivos con aprendizaje automático. No se pretende hacer un estudio estadístico inferencias avanzado sino un estudio exploratorio que muestre hallazgos interesantes en el dataset. Es muy valorable el cómo comuniquéis los hallazgos, por lo que no es cuestión de incluir todos los análisis, sino los que resulten informativos e interesantes, en formatos gráficos o numéricos fáciles de leer y entender.

In [1]:
#****************************************************************************************

## Description of Businesscase - Preventing loss of Early Payment Discount

#### In several countries (e.g. Germany or Austria) the vendors/seller grants a discount when an invoice is paid until a certain date. This depends on the payment conditions agreed between buyer and seller. As an example:
#####  Vendors sends invoice over 1.000 €, if the buyer pays within two weeks, the vendors grants a discount of 2%
[Find out more about EPD](http://www.accountingcoach.com/blog/what-is-an-early-payment-discount)

#### The following scenario and it's solution approach could be of use in order to not miss out on Early Payment Discount (EPD)
##### I've exported a *.csv file from my company's live system of invoice which are posted to the general ledger but not yet paid. 
##### Invoices are paid once the responsible user accepts a workflow, basically saying that the invoice is good to be paid. 
##### In case of invoices for which EPD can be claimed, the user should be advised to action on the workflow in order to retain the agreed percentage over the Grossamount of the invoice. 
##### This notebook could be executed before the weekly payment run of invoices. 

In [2]:
from datetime import timedelta
import numpy, StringIO, datetime
import pandas as pd
import numpy as np

In [3]:
# Importing csv from Accounts Payable tool:
epd = pd.read_csv('PYTEST.csv', sep=';')

In [4]:
epd.head()

,Docnumber,WCprocessor,Email,Baselinedate,Days1,DocDate,FI/MM,Paymentblock,Vendor,Reference,Currency,Gross,Discountamount,Disc.percent1,Termsofpayment,PostingDate,Workflowdescription
0,3323060,KINKHE,test@test.de,28.10.2016,30,28.10.2016,MM,R,1126685,0188/16-AM,EUR,"5.596,80","167,90","3,000",YC24,28.10.2016,Goods Receipt
1,3323058,KINKHE,test@test.de,28.10.2016,30,28.10.2016,MM,R,1126685,0186/16-AM,EUR,"3.935,50","118,07","3,000",YC24,28.10.2016,Goods Receipt
2,3323059,KINKHE,test@test.de,28.10.2016,30,28.10.2016,MM,R,1126685,0187/16-AM,EUR,"3.935,50","118,07","3,000",YC24,28.10.2016,Goods Receipt
3,3323057,KINKHE,test@test.de,27.10.2016,30,27.10.2016,MM,R,1126685,0185/16-AM,EUR,"3.207,60","96,23","3,000",YC24,28.10.2016,Goods Receipt
4,3320693,KINKHE,test@test.de,24.10.2016,10,24.10.2016,MM,R,1093723,9611709159,EUR,"1.942,38","38,85","2,000",YD08,27.10.2016,Goods Receipt


##### Brief explanation of each column

In [5]:
#'Docnumber' - internal number assigned to the invoice
#'WCprocessor' - responsible of approval of invoice (workflow receiver)
#'Email' - user's email address, for confidential reason set to generic value 'test@test.de'
#'Baselinedate' - internal date when EPD time frame starts 
#'Days1' - Days how long EPD can be claimed
#'DocDate' - Invoice date
#'FI/MM' - Not linked to Purchase / linked to Purchaseorder
#'Paymentblock' - Type of paymentblock - at the moment all invoices blocked for payment
#'Vendor' - Vendornumber 
#'Reference' - Invoice number
#'Currency'
#'Gross' - Total amount of invoice
#'Discountamount' - Amount of EPD that can be claimed
#'Disc.percent1' - Percentage of discount over Grossamount
#'Termsofpayment' - internal code for payment conditions
#'PostingDate' - Date of posting against GL
#'Workflowdescription' - Type of workflow sent to WCprocessor

In [6]:
#Converts Baselinedate and Postingdate to datetime
epd.Baselinedate = pd.to_datetime(epd.Baselinedate)
epd.PostingDate = pd.to_datetime(epd.PostingDate)

In [7]:
epd.head()

,Docnumber,WCprocessor,Email,Baselinedate,Days1,DocDate,FI/MM,Paymentblock,Vendor,Reference,Currency,Gross,Discountamount,Disc.percent1,Termsofpayment,PostingDate,Workflowdescription
0,3323060,KINKHE,test@test.de,2016-10-28,30,28.10.2016,MM,R,1126685,0188/16-AM,EUR,"5.596,80","167,90","3,000",YC24,2016-10-28,Goods Receipt
1,3323058,KINKHE,test@test.de,2016-10-28,30,28.10.2016,MM,R,1126685,0186/16-AM,EUR,"3.935,50","118,07","3,000",YC24,2016-10-28,Goods Receipt
2,3323059,KINKHE,test@test.de,2016-10-28,30,28.10.2016,MM,R,1126685,0187/16-AM,EUR,"3.935,50","118,07","3,000",YC24,2016-10-28,Goods Receipt
3,3323057,KINKHE,test@test.de,2016-10-27,30,27.10.2016,MM,R,1126685,0185/16-AM,EUR,"3.207,60","96,23","3,000",YC24,2016-10-28,Goods Receipt
4,3320693,KINKHE,test@test.de,2016-10-24,10,24.10.2016,MM,R,1093723,9611709159,EUR,"1.942,38","38,85","2,000",YD08,2016-10-27,Goods Receipt


### Calculating the days until when EPD can be claimed

In [8]:
# Converting Days1 to string
epd['Days1'] = epd['Days1'].astype(str)

In [9]:
# Adding 'D' to make it recognizeable by pd.to_timedelta
epd["Days1"] = epd["Days1"] + 'D'

In [10]:
#Converting to timedelta
epd["Days1"] = pd.to_timedelta(epd["Days1"].values)

In [11]:
epd.head()

,Docnumber,WCprocessor,Email,Baselinedate,Days1,DocDate,FI/MM,Paymentblock,Vendor,Reference,Currency,Gross,Discountamount,Disc.percent1,Termsofpayment,PostingDate,Workflowdescription
0,3323060,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0188/16-AM,EUR,"5.596,80","167,90","3,000",YC24,2016-10-28,Goods Receipt
1,3323058,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0186/16-AM,EUR,"3.935,50","118,07","3,000",YC24,2016-10-28,Goods Receipt
2,3323059,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0187/16-AM,EUR,"3.935,50","118,07","3,000",YC24,2016-10-28,Goods Receipt
3,3323057,KINKHE,test@test.de,2016-10-27,30 days,27.10.2016,MM,R,1126685,0185/16-AM,EUR,"3.207,60","96,23","3,000",YC24,2016-10-28,Goods Receipt
4,3320693,KINKHE,test@test.de,2016-10-24,10 days,24.10.2016,MM,R,1093723,9611709159,EUR,"1.942,38","38,85","2,000",YD08,2016-10-27,Goods Receipt


In [12]:
#Creating column LIMIT, which shows until when EPD can be claimed. 
#('Baselinedate' - internal date when EPD time frame starts) + (#'Days1' - Days how long EPD can be claimed)
epd["LIMIT"] = epd["Baselinedate"] + epd["Days1"]

#### Rearranging columns, so that LIMIT is the first

In [13]:
cols = epd.columns.tolist()
cols = cols[-1:] + cols[:-1]
epd = epd[cols]

In [14]:
#Now LIMIT is the 'sum' of Baselinedate and Days1
epd.head()

,LIMIT,Docnumber,WCprocessor,Email,Baselinedate,Days1,DocDate,FI/MM,Paymentblock,Vendor,Reference,Currency,Gross,Discountamount,Disc.percent1,Termsofpayment,PostingDate,Workflowdescription
0,2016-11-27,3323060,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0188/16-AM,EUR,"5.596,80","167,90","3,000",YC24,2016-10-28,Goods Receipt
1,2016-11-27,3323058,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0186/16-AM,EUR,"3.935,50","118,07","3,000",YC24,2016-10-28,Goods Receipt
2,2016-11-27,3323059,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0187/16-AM,EUR,"3.935,50","118,07","3,000",YC24,2016-10-28,Goods Receipt
3,2016-11-26,3323057,KINKHE,test@test.de,2016-10-27,30 days,27.10.2016,MM,R,1126685,0185/16-AM,EUR,"3.207,60","96,23","3,000",YC24,2016-10-28,Goods Receipt
4,2016-11-03,3320693,KINKHE,test@test.de,2016-10-24,10 days,24.10.2016,MM,R,1093723,9611709159,EUR,"1.942,38","38,85","2,000",YD08,2016-10-27,Goods Receipt


In [15]:
#Addine today's date
epd["today"] = pd.to_datetime('today')

#### Rearranging columns, so that today comes first

In [16]:
cols = epd.columns.tolist()
cols = cols[-1:] + cols[:-1]
epd = epd[cols]

In [17]:
epd.head()

,today,LIMIT,Docnumber,WCprocessor,Email,Baselinedate,Days1,DocDate,FI/MM,Paymentblock,Vendor,Reference,Currency,Gross,Discountamount,Disc.percent1,Termsofpayment,PostingDate,Workflowdescription
0,2016-10-30,2016-11-27,3323060,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0188/16-AM,EUR,"5.596,80","167,90","3,000",YC24,2016-10-28,Goods Receipt
1,2016-10-30,2016-11-27,3323058,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0186/16-AM,EUR,"3.935,50","118,07","3,000",YC24,2016-10-28,Goods Receipt
2,2016-10-30,2016-11-27,3323059,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0187/16-AM,EUR,"3.935,50","118,07","3,000",YC24,2016-10-28,Goods Receipt
3,2016-10-30,2016-11-26,3323057,KINKHE,test@test.de,2016-10-27,30 days,27.10.2016,MM,R,1126685,0185/16-AM,EUR,"3.207,60","96,23","3,000",YC24,2016-10-28,Goods Receipt
4,2016-10-30,2016-11-03,3320693,KINKHE,test@test.de,2016-10-24,10 days,24.10.2016,MM,R,1093723,9611709159,EUR,"1.942,38","38,85","2,000",YD08,2016-10-27,Goods Receipt


In [18]:
# If today is smaller than LIMIT a mail should be sent to the user, so that EPD is not lost.
# DROP FALSE FROM TABLE

In [19]:
# Adding column 'EPDclaimable' this is the case when 'today' is smaller than LIMIT
epd["EPDclaimable"] = epd.today < epd.LIMIT

#### Rearranging columns, so that EPDclaimable comes first

In [20]:
#Once again rearranging the Dataframe epd
cols = epd.columns.tolist()
cols = cols[-1:] + cols[:-1]
epd = epd[cols]

In [21]:
epd.head()

,EPDclaimable,today,LIMIT,Docnumber,WCprocessor,Email,Baselinedate,Days1,DocDate,FI/MM,Paymentblock,Vendor,Reference,Currency,Gross,Discountamount,Disc.percent1,Termsofpayment,PostingDate,Workflowdescription
0,True,2016-10-30,2016-11-27,3323060,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0188/16-AM,EUR,"5.596,80","167,90","3,000",YC24,2016-10-28,Goods Receipt
1,True,2016-10-30,2016-11-27,3323058,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0186/16-AM,EUR,"3.935,50","118,07","3,000",YC24,2016-10-28,Goods Receipt
2,True,2016-10-30,2016-11-27,3323059,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0187/16-AM,EUR,"3.935,50","118,07","3,000",YC24,2016-10-28,Goods Receipt
3,True,2016-10-30,2016-11-26,3323057,KINKHE,test@test.de,2016-10-27,30 days,27.10.2016,MM,R,1126685,0185/16-AM,EUR,"3.207,60","96,23","3,000",YC24,2016-10-28,Goods Receipt
4,True,2016-10-30,2016-11-03,3320693,KINKHE,test@test.de,2016-10-24,10 days,24.10.2016,MM,R,1093723,9611709159,EUR,"1.942,38","38,85","2,000",YD08,2016-10-27,Goods Receipt


In [22]:
#When value is 'True' the user should be advised to accept workflow and release for payment.
#Then, the EPD can be claimed. 
#Other lines with value 'False' will be dropped
epd = epd[epd.EPDclaimable == True]

In [23]:
#New dataframe created containing only EPDclaimable == True
pd.unique(epd["EPDclaimable"].values.ravel())

array([True], dtype=object)

#### Now, the data is prepared so that the user, who is still pending to approve the workflow, can be contacted.  
#### A workflow could also be rejected for exmaple in case of overpricing. Anyhow, an email should be sent to every user to inform about possible loss of EPD. The user will be informed about:
* The date until when the invoice should be approved
* The discountamount which could be claimed.


In [24]:
# Creating smaller dataframe to test email sending

In [25]:
epd = epd[0:2]

In [26]:
epd

,EPDclaimable,today,LIMIT,Docnumber,WCprocessor,Email,Baselinedate,Days1,DocDate,FI/MM,Paymentblock,Vendor,Reference,Currency,Gross,Discountamount,Disc.percent1,Termsofpayment,PostingDate,Workflowdescription
0,True,2016-10-30,2016-11-27,3323060,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0188/16-AM,EUR,"5.596,80","167,90","3,000",YC24,2016-10-28,Goods Receipt
1,True,2016-10-30,2016-11-27,3323058,KINKHE,test@test.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0186/16-AM,EUR,"3.935,50","118,07","3,000",YC24,2016-10-28,Goods Receipt


In [27]:
# For purposes of testing
epd["Email"][0] = str(raw_input("Please, type in your test email address "))

Please, type in your test email address markwellings@gmx.de


In [28]:
epd["Email"][1] = 'markwellings@campusciff.net'

C:\Users\Mark\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [29]:
epd

,EPDclaimable,today,LIMIT,Docnumber,WCprocessor,Email,Baselinedate,Days1,DocDate,FI/MM,Paymentblock,Vendor,Reference,Currency,Gross,Discountamount,Disc.percent1,Termsofpayment,PostingDate,Workflowdescription
0,True,2016-10-30,2016-11-27,3323060,KINKHE,markwellings@gmx.de,2016-10-28,30 days,28.10.2016,MM,R,1126685,0188/16-AM,EUR,"5.596,80","167,90","3,000",YC24,2016-10-28,Goods Receipt
1,True,2016-10-30,2016-11-27,3323058,KINKHE,markwellings@campusciff.net,2016-10-28,30 days,28.10.2016,MM,R,1126685,0186/16-AM,EUR,"3.935,50","118,07","3,000",YC24,2016-10-28,Goods Receipt


In [30]:
# Changing format of LIMIT to Day, Month, Year. 
epd['LIMIT'] = epd['LIMIT'].dt.strftime('%d/%m/%Y')

In [31]:
import smtplib # importing library to email sending

In [32]:
import smtplib

def sendmail(): # creating definition
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText

    # me == my email address
    # you == recipient's email address
    me = "Mark Wellings"
    you = email # has to be user mail adress from table
    limit_mail = limit
    discamount_mail = discamount
    docnumber_mail = docnum

    # Create message container 
    msg = MIMEMultipart('alternative')
    msg['Subject'] = "Loss of Early Payment Discount"
    msg['From'] = me
    msg['To'] = you

    # Create the body of the message.
    html = """
        <html>
            <body>
            <i><h3>Possible loss of Early Payment Discount</h3></i>
            Hi! <p>
            Please, take action on invoice with docnumber <b>%s</b> it has to be approved until <em>%s</em> to 
            claim the Early Payment Discount of <strong>%s %s</strong>. <p>
            Thanks very much for your collaboration!
            <hr>
            </body>
        </html>
        """ %(docnum, limit, discamount, curr)

    # Record the message
    part1 = MIMEText(html, 'html')
         

    # Attach parts into message container.   
    msg.attach(part1)
        
    # Send the message via local SMTP server.
    mail = smtplib.SMTP('smtp.gmail.com', 587)
    
    mail.ehlo()

    mail.starttls()

    mail.login('pythontest123new@gmail.com', 'TEST.TEST.123') # Testaccount from which the email is sent
    mail.sendmail(me, you, msg.as_string())
    mail.quit()

In [33]:
# Interrating over dataframe and executing definition

In [34]:
email = epd['Email'].values[0]
limit = epd['LIMIT'].values[0]
discamount = epd["Discountamount"].values[0]
docnum = epd["Docnumber"].values[0]
curr = epd["Currency"].values[0]

for index, row in epd.iterrows():
    email
    limit
    discamount
    sendmail()
    email = epd['Email'].values[0+1]
    limit = epd['LIMIT'].values[0+1]
    discamount = epd["Discountamount"].values[0+1]
    docnum = epd["Docnumber"].values[0+1]
    curr = epd["Currency"].values[0+1]

### Possible improvements
* Connecting directly to DB to download file


### Further possible analisis on data could be:
* a statiscal analysis based on several criteria like vendor, workflow description or posting date
* a machine learning model which releases payment blocks